# Logging

To keep track of what is happening in your code, you can use the provided logger instance:

In [1]:
from qsprpred.logs import logger

logger.warning("This is a warning.")

qsprpred - WARNING - /tmp/ipykernel_12065/1394893261.py:3 - This is a warning.


You can see the logger displays its name, the level of the message, the current file with a line number and the message itself. This logger is used throughout the codebase to keep track of what is happening, but feel free to use it for your own code as well just like in the example above. You can set the message level to `debug`, `info`, `warning`, `error` or `critical`. The default level is `warning`, so `debug` and `info` messages will not be displayed. This is useful to know when you want to debug a piece of QSPRpred code:

In [2]:
import os
from qsprpred.data import QSPRDataset

os.makedirs("../../tutorial_output/data", exist_ok=True)
QSPRDataset.fromTableFile(  # no message is displayed
    filename='../../tutorial_data/A2A_LIGANDS.tsv',
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

# change logger for one with a lower level
from qsprpred.logs import setLogger
import logging

logger.setLevel(logging.DEBUG)
setLogger(logger)

QSPRDataset.fromTableFile(  # more information is shown
    filename='../../tutorial_data/A2A_LIGANDS.tsv',
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

/home/sichom/projects/QSPRpred/qsprpred/data/tables/mol.py:160: UserWarning: Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.
  super().__init__(
/home/sichom/projects/QSPRpred/qsprpred/data/tables/mol.py:160: UserWarning: Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.
  super().__init__(
qsprpred - DEBUG - /home/sichom/projects/QSPRpred/qsprpred/data/tables/qspr.py:1045 - Adding target property 'pchembl_value_Mean' to dataset.
qsprpred - DEBUG - /home/sichom/projects/QSPRpred/qsprpred/data/tables/qspr.py:264 - Restoring training data...
qsprpred - DEBUG - /home/sichom/projects/QSPRpred/qsprpred/data/tables/qspr.py:279 - Training data restored.
qsprpred - DEBUG - /home/sichom/projects/QSPRpred/qsprpred/data/tables/qspr.py:280 - Training features shape: (3265, 0)
qsprpred - DEBUG - /h

In [3]:
# set the level back to warnings only

logger.setLevel(logging.WARNING)
setLogger(logger)

The `enable_file_logger` utility function will
create a file logger that will write to the file `TutorialLogging.log` in the `tutorial_output` directory.

In [4]:
# logging settings
from qsprpred.logs.utils import enable_file_logger

logSettings = enable_file_logger(
    log_folder='../../tutorial_output/',
    filename='TutorialLogging.log',
    debug=False,
    disable_existing_loggers=False
)

A utility function is provided to save your conda environment to a file. This is useful if you want to share your code with others, or if you want to recreate your environment at a later time.

In [5]:
from qsprpred.logs.utils import export_conda_environment

# save the environment to a yaml file
export_conda_environment("../../tutorial_output/TutorialLoggingEnvironment.yml")

Environment exported to ../../tutorial_output/TutorialLoggingEnvironment.yml successfully!


Below we run a standard data preparation and modelling workflow to show logging output.

In [6]:
from qsprpred.data import RandomSplit
from qsprpred.data.descriptors.sets import FingerprintSet
from qsprpred.data.descriptors.calculators import MoleculeDescriptorsCalculator
from qsprpred.data import QSPRDataset
# Standard data preparation
import os

os.makedirs("../../tutorial_output/data", exist_ok=True)

dataset = QSPRDataset.fromTableFile(
    filename='../../tutorial_data/A2A_LIGANDS.tsv',
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

feature_calculator = MoleculeDescriptorsCalculator(
    desc_sets=[FingerprintSet(fingerprint_type="MorganFP", radius=3, nBits=2048)]
)

dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=[feature_calculator],
)

dataset.save()

/home/sichom/projects/QSPRpred/qsprpred/data/tables/mol.py:160: UserWarning: Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.
  super().__init__(


In [7]:
# Standard model training
from qsprpred.models import SklearnModel
from sklearn.neighbors import KNeighborsRegressor
from qsprpred.models import CrossValAssessor, TestSetAssessor

os.makedirs("../../tutorial_output/models", exist_ok=True)

model = SklearnModel(
    base_dir='../../tutorial_output/models',
    data=dataset,
    alg=KNeighborsRegressor,
    name='TutorialLoggingModel'
)

CrossValAssessor('r2')(model)
TestSetAssessor('r2')(model)

model.fitAttached()

model.save()

/home/sichom/software/miniconda/envs/qsprpred/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/sichom/software/miniconda/envs/qsprpred/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/sichom/software/miniconda/envs/qsprpred/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/sichom/software/miniconda/envs/qsprpred/lib/python3.10/site-packages/sklearn/utils/validation.py:767: Future

'/home/sichom/projects/QSPRpred/tutorials/tutorial_output/models/TutorialLoggingModel/TutorialLoggingModel_meta.json'